In [29]:
%load_ext autoreload
%autoreload 2


from util_0630 import *
from myalgorithm_0630_1 import algorithm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- 실험해볼 내용들

1. 가까운 거리는 먼저 WALK를 배정하면 어떨까? -> try_merging_bundles_by_dist_walk_prefered 함수 제작 후 실험 결과 결과가 꼭 개선된다는 보장이 없었음
2. 배달원을 전부 한번에 재할당해보면 어떨까? -> check_reassign_riders 함수 제작 후 실험 결과 기존의 배달원 재할당 코드보다 확실히 효과적이었음
3. 가까운 번들 내의 원소를 교환하는 휴리스틱 코드를 만들면 어떨까?
4. 만약 번들의 각 주문을 CAR로 나누어 할당하였을 때 더 비용이 최소화가 된다면 나누는 언번들링 코드를 제작해보면 어떨까?

- 코드 구조상 개선할 부분

1. get_init_bundle 함수를 util로 옮기기 -> 완료
2. 배달원 커스텀 할당 기능을 util로 옮기기 -> 완료

In [48]:

problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

## --------------- 초기 번들링 최적화 코드 --------------------------

inf = float('inf')

car_rider = ALL_RIDERS[2]

init_availables = [rider.available_number for rider in ALL_RIDERS]

min_init_cost = inf
min_init_cost_bundle = []
min_init_cost_rider_availables = []

# try_merging_bundles_by_dist // 모든 배달원과 모든 조합에서의 최소 거리를 보는 케이스
# try_merging_bundles_by_dist_walk_prefered // 위와 동일하지만 우선적으로 도보 배달원을 할당하는 케이스
# try_merging_bundles_by_cost // 번들 합칠 때 거리가 아닌 비용을 보는 케이스

for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, try_merging_bundles_by_dist)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles


## ----------------- 기본 배달원 재할당 코드 -------------------

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1


## ----------------- 커스텀 배달원 재할당 코드 -----------------------

all_bundles, rider_availables = check_reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


3608.062000000001
